In [1]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras

from nnfs.datasets import spiral_data

2022-05-23 14:46:09.965535: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 14:46:09.965559: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [40]:
(X_train, y_train), (X_test, y_test) = spiral_data(samples=5000, classes=3), spiral_data(samples=500, classes=3)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [67]:
def hyper_model(hp):
    units = hp.Int(name="units_layer_1", min_value=16, max_value=512, step=16)
    
    inputs = keras.Input(shape=(2, ))
    x = keras.layers.Dense(units, activation="relu", 
                           kernel_regularizer=keras.regularizers.L2(5e-4), 
                           bias_regularizer=keras.regularizers.L2(5e-4))(inputs)
    x = keras.layers.Dense(units, activation="relu", 
                           kernel_regularizer=keras.regularizers.L2(5e-4), 
                           bias_regularizer=keras.regularizers.L2(5e-4))(x)
    x = keras.layers.Dense(3, activation="softmax")(x)
    model = keras.Model(inputs, x)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [69]:
tuner = kt.BayesianOptimization(
    hyper_model,
    objective="val_accuracy",
    max_trials=10,
    overwrite=True
)

In [70]:
tuner.search(X_train, y_train, batch_size=512, epochs=100, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 12s]
val_accuracy: 0.8939999938011169

Best val_accuracy So Far: 0.9013333320617676
Total elapsed time: 00h 02m 55s
INFO:tensorflow:Oracle triggered exit


In [71]:
model = tuner.get_best_models()